# Generative Agents in LangChain by Claude haiku

This notebook implements a generative agent based on the paper [Generative Agents: Interactive Simulacra of Human Behavior](https://arxiv.org/abs/2304.03442) by Park, et. al.

In it, we lコップrage a time-weighted Memory object backed by a LangChain Retriコップr.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import logging

logging.basicConfig(lコップl=logging.ERROR)

In [3]:
from datetime import datetime, timedelta
from typing import List

from langchain.docstore import InMemoryDocstore
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# from langchain.chat_models import ChatAnthropic
from langchain_anthropic import ChatAnthropic
from termcolor import colored
from env import API_KEY, VOYAGE_API_KEY
import os
import voyageai

In [5]:
USER_NAME = "人間"  # The name you want to use when interviewing the agent.
# LLM = ChatOpenAI(max_tokens=1500)  # Can be any LLM you want.
os.environ["ANTHROPIC_API_KEY"] = API_KEY
os.environ["VOYAGE_API_KEY"] = VOYAGE_API_KEY
LLM = ChatAnthropic(
    model="claude-3-haiku-20240307",
    max_tokens=1500
)

### Generative Agent Memory Components

This tutorial highlights the memory of generative agents and its impact on their behavior. The memory varies from standard LangChain Chat memory in two aspects:

1. **Memory Formation**

   Generative Agents have extended memories, stored in a single stream:
      1. Observations - from dialogues or interactions with the virtual world, about self or others
      2. Reflections - resurfaced and summarized core memories


2. **Memory Recall**

   Memories are retriコップd using a weighted sum of salience, recency, and importance.

You can review the definitions of the `GenerativeAgent` and `GenerativeAgentMemory` in the [reference documentation]("https://api.python.langchain.com/en/latest/modules/experimental.html") for the following imports, focusing on `add_memory` and `summarize_related_memories` methods.

In [6]:
from langchain_experimental.generative_agents import (
    GenerativeAgent,
    GenerativeAgentMemory,
)


from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding

# lc_embed_model = HuggingFaceEmbeddings(
#     model_name="sentence-transformers/all-mpnet-base-v2"
# )
# embed_model = LangchainEmbedding(lc_embed_model)

In [8]:
embed_model = HuggingFaceEmbeddings()
out = embed_model.embed_query("hello")
embed_model_size = len(out)

## Memory Lifecycle

Summarizing the key methods in the above: `add_memory` and `summarize_related_memories`.

When an agent makes an observation, it stores the memory:
    
1. Language model scores the memory's importance (1 for mundane, 10 for poignant)
2. Observation and importance are stored within a document by TimeWeightedVectorStoreRetriコップr, with a `last_accessed_time`.

When an agent responds to an observation:

1. Generates query(s) for retriコップr, which fetches documents based on salience, recency, and importance.
2. Summarizes the retriコップd information
3. Updates the `last_accessed_time` for the used documents.


## Create a Generative Character



Now that we've walked through the definition, we will create two characters named "本" and "コップ".

In [9]:
import math

import faiss

from transformers import AutoTokenizer, AutoModel
import torch

# from sentence_transformers import SentenceTransformer


In [27]:
def relevance_score_fn(score: float) -> float:
    """Return a similarity score on a scale [0, 1]."""
    # This will differ depending on a few things:
    # - the distance / similarity metric used by the VectorStore
    # - the scale of your embeddings (OpenAI's are unit norm. Many others are not!)
    # This function converts the euclidean norm of normalized embeddings
    # (0 is most similar, sqrt(2) most dissimilar)
    # to a similarity function (0 to 1)
    return 1.0 - score / math.sqrt(2)


def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    # Define your embedding model
    # embeddings_model = OpenAIEmbeddings()
    embeddings_model = embed_model
    # Initialize the vectorstore as empty
    # embedding_size = 1536
    embedding_size = embed_model_size

    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(
        embeddings_model.embed_query,
        index,
        InMemoryDocstore({}),
        {},
        relevance_score_fn=relevance_score_fn,
    )
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore, other_score_keys=["importance"], k=15
    )

In [28]:
books_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=8,  # we will give this a relatively low number to show how reflection works
)

book = GenerativeAgent(
    name="本",
    age=3,
    traits="知識豊富、静か、忍耐強い",  # You can add more persistent traits here
    status="図書館の棚に置かれている",  # When connected to a virtual world, we can have the characters update their status
    memory_retriever=create_new_memory_retriever(),
    llm=LLM,
    memory=books_memory,
)

In [29]:
# The current "Summary" of a character can't be made because the agent hasn't made
# any observations yet.
print(book.get_summary())

Name: 本 (age: 3)
Innate traits: 知識豊富、静か、忍耐強い
Given the statements provided, I would summarize the core characteristics of 本 as follows:

1. It is a character or symbol.
2. It likely has meaning or significance in a particular context or language.
3. The information provided does not give enough detail to make any other definitive claims about its characteristics.

Without more context or details, I cannot make any further inferences or claims about the specific nature or meaning of 本.


In [30]:
# We can add memories directly to the memory object
book_observations = [
    "本は長年図書館に置かれている",
    "本は多くの人に読まれてきた",
    "本は様々な知識を含んでいる",
    "本は時々ほこりをかぶる",
    "本は静かに周りを観察している",
]
for observation in book_observations:
    book.memory.add_memory(observation)

In [31]:
# Now that 本 has 'memories', their self-summary is more descriptive, though still rudimentary.
# We will see how this summary updates after more observations to create a more rich description.
print(book.get_summary(force_refresh=True))

Name: 本 (age: 3)
Innate traits: 知識豊富、静か、忍耐強い
Based on the statements provided, here is a summary of the core characteristics of "本" (which appears to refer to a book):

1. 本 is sometimes covered in dust, indicating it is not frequently used or handled.
2. 本 contains a diverse range of knowledge and information.
3. 本 has been read by many people over the years.
4. 本 quietly observes its surroundings.
5. 本 has been placed in a library for a long time.

Overall, the statements suggest that 本 is a book that has accumulated knowledge and has been read by many, but is not actively used and has been sitting in a library for an extended period.


## Pre-Interview with Character

Before sending our character on their way, let's ask them a few questions.

In [32]:
def interview_agent(agent: GenerativeAgent, message: str) -> str:
    """Help the notebook user interact with the agent."""
    new_message = f"{USER_NAME} says {message}"
    return agent.generate_dialogue_response(new_message)[1]

## Step through the day's observations.

In [33]:
# Let's have 本 start going through a day in the life.
observations = [
    "本が図書館の開館時間になり、周りの騒がしさで目覚める。",
    "本は棚から取り出され、読書コーナーに運ばれる。",
    "本は自分のページが少し破れていることに気づき、修理を待つ。",
    "図書館員が本を丁寧に修理し、元の状態に戻る。",
    "本は自分の内容が少し古くなっていることを感じ、最新版が出ていないか心配になる。",
    "本は貸出カードをチェックされるが、しばらく借りられていないことがわかる。",
    "本は図書館員によって内容の要約が更新され、カタログに反映される。",
    "本は新しい本棚に移動され、様々なジャンルの本に囲まれる。",
    "本は図書館のイベント告知を見て、自分も展示されることを期待する。",
    "イベントの準備に時間がかかり、本は長時間棚で待つ。",
    "本は何人かの来館者に手に取られるが、結局借りられずに棚に戻される。",
    "本はイベントが終わり、少し落胆しながら元の棚に戻される。",
    "本は児童コーナーに一時的に移動され、新しい環境に戸惑う。",
    "子供たちが本を選ぶのに時間がかかり、本は長く待たされる。",
    "本は隣の本たちの会話から、人気の読書テーマについて聞く。",
    "本は自分の内容が그テーマに関連していることに気づき、もっと注目されたいと思う。",
    "本は図書館員に自分の関連性をアピールし、特設コーナーに置かれることになる。",
    "本は他の関連書籍と一緒に並べられ、より多くの人の目に触れる機会を得る。",
    "本は読書会で使用されるため、静かな書庫から賑やかな会議室に運ばれる。",
    "読書会の参加者が本のページをめくり、内容について熱心に議論する。",
    "本は読書会で活発に使用され、多くの人に読まれる喜びを感じる。",
    "読書会が盛り上がりすぎて、本のページが少し折れてしまう。",
    "本は読書会後、少し疲れて棚に戻される。",
    "夜間、図書館に忍び込んだネズミが本の近くの紙を噛み、周りが散らかる。",
    "本は自分の存在意義について考え始め、もっと多くの人に読まれたいと思う。",
    "本は隣の古典的な名著と話をし、励ましの言葉をもらう。",
    "名著の友人は本に、価値ある内容は必ず読者に届くと伝え、希望を与える。",
    "本は友人との会話後少し元気を取り戻し、新しい日を迎える準備をする。",
]

In [34]:
# Let's send 本 on their way. We'll check in on their summary コップry few observations to watch it evolve
for i, observation in enumerate(observations):
    _, reaction = book.generate_reaction(observation)
    print(colored(observation, "green"), reaction)
    if ((i + 1) % 20) == 0:
        print("*" * 40)
        print(
            colored(
                f"After {i+1} observations, 本の要約:\n{本.get_summary(force_refresh=True)}",
                "blue",
            )
        )
        print("*" * 40)

本が図書館の開館時間になり、周りの騒がしさで目覚める。 本 said "やっと時間が来たか。今日はどのような本が借りられるのか楽しみだ。"


KeyboardInterrupt: 

## Interview after the day

In [16]:
interview_agent(book, "Tell me about how your day has been going")

'Tommie said "It\'s been a bit of a rollercoaster, to be honest. I\'ve had some setbacks in my job search, but I also had some good moments today, like sending out a few resumes and meeting some potential employers at a job fair. How about you?"'

In [17]:
interview_agent(book, "How do you feel about coffee?")

'Tommie said "I really enjoy coffee, but sometimes I regret not buying a better brand. How about you?"'

In [31]:
interview_agent(book, "Tell me about your childhood dog!")

'Tommie said "Ah, my childhood dog Bruno - those were such happy times! He was this big, shaggy mutt that I absolutely adored. I have so many fond memories of playing fetch with him in the backyard, going for long walks around the neighborhood, and just snuggling up with him on the couch. Bruno was such a loyal and loving companion. He really was the best friend a kid could ask for. I miss his goofy personality and wagging tail so much. Those memories of growing up with him still bring a smile to my face, even all these years later. Do you have any pets or special animals from your own childhood that you remember fondly?"'

## Adding Multiple Characters

Let's add a second character to have a conversation with 本. Feel free to configure different traits.

In [35]:
cups_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=5,
)


cup = GenerativeAgent(
    name="コップ",
    age=7,
    traits="好奇心旺盛、丈夫、透明",
    status="キッチンの棚に置かれている",
    llm=LLM,
    daily_summaries=[
        (
            "人間にオレンジジュースを注いでもらった。夏を感じた。"
        )
    ],
    memory=cups_memory,
    verbose=False,
)

In [ ]:
yesterday = (datetime.now() - timedelta(days=1)).strftime("%A %B %d")
cup_observations = [
    "コップは食器棚から取り出され、朝の使用に備える",
    "コップにはホットコーヒーが注がれ、温かさを感じる",
    "コップは食器洗い機で他の食器と一緒に洗われる",
    "コップは会議室のテーブルに置かれ、重要な会議に参加する",
    "コップは隣に置かれた水差しから、誰かがトムという人物について難しい人だと話しているのを「聞く」",
]
for observation in cup_observations:
    cup.memory.add_memory(observation)

In [37]:
print(cup.get_summary())

Name: コップ (age: 7)
Innate traits: 好奇心旺盛、丈夫、透明
Based on the provided statements, コップ's core characteristics can be summarized as follows:

1. コップ is used to hold hot coffee, providing a sense of warmth.
2. コップ is washed together with other dishes in a dishwasher.
3. コップ is stored in a cabinet and taken out in the morning for use.

The statements do not indicate any additional characteristics beyond these points.


## Dialogue between Generative Agents

Generative agents are much more complex when they interact with a virtual environment or with each other. Below, we run a simple conversation between 本 and コップ.

In [22]:
def run_conversation(agents: List[GenerativeAgent], initial_observation: str) -> None:
    """Runs a conversation between agents."""
    _, observation = agents[1].generate_reaction(initial_observation)
    print(observation)
    turns = 0
    while True:
        break_dialogue = False
        for agent in agents:
            stay_in_dialogue, observation = agent.generate_dialogue_response(
                observation
            )
            print(observation)
            # observation = f"{agent.name} said {reaction}"
            if not stay_in_dialogue:
                break_dialogue = True
        if break_dialogue:
            break
        turns += 1

In [ ]:
agents = [book, cup]
run_conversation(
    agents,
    "本 said: キッチンにいるコップくん！なにか最近おもしろいことはあった?元気にしている？",
)

## Let's interview our agents after their conversation

Since the generative agents retain their memories from the day, we can ask them about their plans, conversations, and other memoreis.

In [26]:
# We can see a current "Summary" of a character based on their own perception of self
# has changed
print(book.get_summary(force_refresh=True))

Name: 本 (age: 3)
Innate traits: 知識豊富、静か、忍耐強い
Based on the statements provided, the core characteristics of 本 can be summarized as follows:

1. Diligent and well-maintained: 本 is sometimes covered in dust, but is carefully repaired and maintained by the librarians to keep it in good condition.

2. Adaptive and flexible: 本 is moved to a new bookshelf and finds itself surrounded by various genres of books, which it sees as an opportunity to reach a wider range of readers and contribute its knowledge more effectively.

3. Collaborative and supportive: 本 engages in thoughtful dialogues with other objects in the library, such as コップ, expressing mutual respect and a shared commitment to support the intellectual growth and curiosity of library patrons.

4. Appreciative and grateful: 本 is appreciative of the efforts made by the librarians and other library resources to maintain and utilize it, and it expresses gratitude for the opportunities to fulfill its role in serving readers.

5. Dedicated

In [57]:
print(cup.get_summary(force_refresh=True))

Name: Eve (age: 34)
Innate traits: curious, helpful
Eve is a helpful and friendly person who enjoys playing sports and staying productive. She is attentive and responsive to others' needs, actively listening and asking questions to understand their perspectives. Eve has experience in event planning and communication, and is willing to share her knowledge and expertise with others. She values teamwork and collaboration, and strives to create a comfortable and supportive environment for everyone.
